# Banca d'Italia - depositi e impieghi

In [33]:
import sqlite3
import pandas as pd
conn = sqlite3.connect("D:/files/Bankit.sqlite")
cursor = conn.cursor()

In [3]:
query = "SELECT * FROM TFR10194"
TFR10194 = pd.read_sql_query(query, conn)

In [16]:
TFR10194.query('PRV_SPORT == "ITC4D" \
and DATA_OSS == "2017-12-31 00:00:00.000000" \
and LOC_SPORT == 9999997 \
and FENEC == "1041810"')

,DATA_OSS,DIVISA1,DURORI,ENTE_SEGN,FENEC,LOC_SPORT,PRV_SPORT,RESIDENZA1,VALORE,STATUS
94225,2017-12-31 00:00:00.000000,1000,9,1100010,1041810,9999997,ITC4D,IT,19076556,NaN


## Prestiti (esclusi PCT), depositi (esclusi PCT) e numero sportelli - per comune dello sportello [TFR10194]
- contiene depositi impeghi e sportelli per comune
- ha frequenza annuale
- è fermo al 2023 : al 31/12 gli impieghi (1041810) Italia (IT) erano **1546906651** Mln€
- Nella [nota pubblicata dal Comune di Monza](https://www.sistan.it/index.php?id=88&no_cache=1&tx_ttnews%5Btt_news%5D=7178) si legge che a fine 2017 i depositi bancari nel territorio ammontano a 19.076,556 milioni di euro, con un incremento del 7,4% rispetto al 2016, mentre gli impieghi scendono nello stesso periodo del 1,4%.Il deposito medio per abitante è di 21.883 euro e la città con deposito pro-capite più elevato è Monza, con con 46.945 euro, seguita da Vimercate con 34.389 euro.


In [34]:
tabella = "TFR10194"
query_sql = f"SELECT * FROM {tabella};"
df = pd.read_sql_query(query_sql, conn)

In [45]:
DATA_OSS = ["2017-12-31 00:00:00.000000", "2023-12-31 00:00:00.000000"]
LOC_SPORT = [9999997, 8888888]  # Più valori numerici
FENEC = ["1041810"]  # Lista con più valori
TFR10194.query("PRV_SPORT in @PRV_SPORT and DATA_OSS in @DATA_OSS and LOC_SPORT in @LOC_SPORT and FENEC in @FENEC")


,DATA_OSS,DIVISA1,DURORI,ENTE_SEGN,FENEC,LOC_SPORT,PRV_SPORT,RESIDENZA1,VALORE,STATUS
8480,2023-12-31 00:00:00.000000,1000,9,1100010,1041810,9999997,IT,IT,1546906651,NaN
96837,2017-12-31 00:00:00.000000,1000,9,1100010,1041810,9999997,IT,IT,1224423827,NaN


In [47]:
# oppure
df["DATA_OSS"] = pd.to_datetime(df["DATA_OSS"], errors='coerce')
PRV_SPORT = ["ITH51"]  # Lista di valori
LOC_SPORT = [9999997]  # Lista numerica
FENEC = ["1041810", "1077778"]  # Lista di valori
fine_trimestre = ["03-31", "06-30", "09-30", "12-31"]
df_filtered = df[
    (df["PRV_SPORT"].isin(PRV_SPORT)) &
    (df["LOC_SPORT"].isin(LOC_SPORT)) &
    (df["FENEC"].isin(FENEC)) &
    (df["DATA_OSS"].dt.strftime("%m-%d").isin(fine_trimestre))]
df_filtered.query("DATA_OSS == @pd.to_datetime('2016-12-31')") # df_filtered.query('DATA_OSS == "2023-12-31"')

,DATA_OSS,DIVISA1,DURORI,ENTE_SEGN,FENEC,LOC_SPORT,PRV_SPORT,RESIDENZA1,VALORE,STATUS
114573,2016-12-31,1000,9,1100010,1077778,9999997,ITH51,IT,6528761,NaN
114989,2016-12-31,1000,9,1100010,1041810,9999997,ITH51,IT,5977457,NaN


## TFR20232

In [67]:
# finanziamenti solo imprese
tabella = "TFR20232"
set_ctp_values = ["SBI25", "S11","S14BI2","SBI42","SBI59","600"]  # Se ci sono più valori, puoi aggiungerli qui
date_fine_trimestre = ['03-31', '06-30', '09-30', '12-31']
loc_ctp_values = ['IT', 'ITC', 'ITF', 'ITF1', 'ITF2', 'ITF3', 'ITF4', 'ITF5', 'ITF6', 'ITG', 'ITG1', 'ITG2', 'ITH', 'ITI']
ateco_ctp_values = ["1004999","1005009"]

query = f"""
    SELECT DATA_OSS, LOC_CTP, SET_CTP, VALORE 
    FROM {tabella}
    WHERE SET_CTP IN ({','.join(f"'{v}'" for v in set_ctp_values)})
    AND strftime('%m-%d', DATA_OSS) IN ({','.join(f"'{v}'" for v in date_fine_trimestre)})
    AND LOC_CTP IN ({','.join(f"'{v}'" for v in loc_ctp_values)})
    AND ATECO_CTP IN ({','.join(f"'{v}'" for v in ateco_ctp_values)})
    ORDER BY 1 DESC;
    """

TFR20232 = pd.read_sql_query(query, conn)
TFR20232


,DATA_OSS,LOC_CTP,SET_CTP,VALORE
0,2024-09-30 00:00:00.000000,ITF1,SBI25,9652667
1,2024-09-30 00:00:00.000000,ITF,SBI42,176194639
2,2024-09-30 00:00:00.000000,ITG,SBI42,81214528
3,2024-09-30 00:00:00.000000,ITH,S14BI2,17783796
4,2024-09-30 00:00:00.000000,ITG1,SBI25,18072535
...,...,...,...,...
3061,2011-03-31 00:00:00.000000,ITC,S14BI2,29642694
3062,2011-03-31 00:00:00.000000,ITG2,S11,11042336
3063,2011-03-31 00:00:00.000000,ITF4,S11,23060255
3064,2011-03-31 00:00:00.000000,ITF3,SBI25,39392164
